In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO




### 2021년 2월 15일 하루 키워드별 ctr

In [39]:
q = """
SELECT i.ref_term,
       count(DISTINCT i.content_id) AS imp_pcnt,
       count(i.imp_id) AS imp_cnt,
       count(c.imp_id) AS click_cnt,
       click_cnt/imp_cnt::float AS ctr
FROM
  (SELECT imp_id,
          content_id,
          trim(ref_term) AS ref_term
   FROM bun_log_db.api_event_type_impression_ad
   WHERE YEAR||MONTH||DAY = '20210215'
     AND page_id = '검색결과') i
LEFT JOIN
  (SELECT imp_id
   FROM bun_log_db.api_event_type_click_ad
   WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
GROUP BY 1
"""
daily = pd.read_sql(q, con=bun_dw)

In [22]:
daily.shape

(124677, 5)

In [40]:
daily.describe()

,imp_pcnt,imp_cnt,click_cnt,ctr
count,124677.000000,124677.000000,124677.000000,124677.000000
mean,7.012953,26.727953,0.651227,0.032035
std,11.976435,306.223441,9.148504,0.133377
min,1.000000,1.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000
50%,3.000000,4.000000,0.000000,0.000000
75%,7.000000,11.000000,0.000000,0.000000
max,400.000000,40064.000000,1264.000000,1.000000


In [41]:
daily.quantile([0.8, 0.9, 0.95, 0.99])

,imp_pcnt,imp_cnt,click_cnt,ctr
0.80,9.0,15.0,0.0,0.000000
0.90,17.0,31.0,1.0,0.047619
0.95,28.0,58.0,2.0,0.166667
0.99,58.0,304.0,9.0,1.000000


In [35]:
q = '''
SELECT i.ref_term,
       i.ad_set_product_id,
       count(i.imp_id) AS imp_cnt,
       count(c.imp_id) AS click_cnt,
       click_cnt/imp_cnt::float AS ctr
FROM
  (SELECT imp_id,
          ad_set_product_id,
          trim(ref_term) AS ref_term
   FROM bun_log_db.api_event_type_impression_ad
   WHERE YEAR||MONTH||DAY = '20210215'
     AND page_id = '검색결과') i
LEFT JOIN
  (SELECT imp_id
   FROM bun_log_db.api_event_type_click_ad
   WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
GROUP BY 1, 2
'''

keyword_product_ctr = pd.read_sql(q, con=bun_dw)

In [39]:
df = pd.merge(daily, keyword_product_ctr.groupby('ref_term', as_index=False)['ctr'].mean(), on='ref_term')
df.columns = ['ref_term', 'imp_pcnt', 'imp_cnt', 'click_cnt', 'keyword_ctr', 'mean_keyword_product_ctr']

In [5]:
# df.to_csv("keyword_ctr.csv", index=False)
# df = pd.read_csv('keyword_ctr.csv')

In [21]:
q = '''
SELECT trim(search_term) AS ref_term,
       count(*) AS search_cnt
FROM bun_log_db.api_event_type_search
WHERE YEAR||MONTH||DAY = '20210215'
GROUP BY 1
'''
search = pd.read_sql(q, con=bun_dw)

In [23]:
search.head()

,ref_term,search_cnt
0,알파바운스,203
1,더보이즈 트레카,15
2,리본 브릿지,1
3,유리 렌티큘러,1
4,아디다스275,16


In [22]:
daily_nonad.head(10)

,ref_term,imp_nonad_pcnt,non_ad_imp_cnt,non_ad_click_cnt,non_ad_ctr
0,!i5 9400,264,306,2,0.006536
1,!국내발송!,48,71,2,0.028169
2,!국내발송! 샤넬,53,57,10,0.175439
3,#235,59,62,4,0.064516
4,#K7,119,122,1,0.008197
5,#gt730,4,4,1,0.250000
6,#강혜원ar,13,14,1,0.071429
7,#검더텅,5,5,0,0.000000
8,#고양시화정,24,27,0,0.000000
9,#공유,14,15,0,0.000000


In [30]:
# df = pd.merge(df, search, on='ref_term')
# df = pd.merge(df, daily_nonad[['ref_term', 'imp_nonad_pcnt']], on='ref_term')
# df = df[['ref_term', 'search_cnt', 'imp_pcnt', 'imp_nonad_pcnt', 'mean_keyword_product_ctr', 'imp_cnt', 'click_cnt', 'keyword_ctr']]
df.to_csv('df.csv', index=False)

In [41]:
#impression count가 상위 5% 안에 들어가는 키워드 vs 안들어가는 키워드
top = df[df['imp_cnt'] >= df['imp_cnt'].quantile(0.95)]
no_top = df[df['imp_cnt'] < df['imp_cnt'].quantile(0.95)]

In [43]:
top.describe()

,imp_pcnt,imp_cnt,click_cnt,keyword_ctr,mean_keyword_product_ctr
count,6365.000000,6365.000000,6365.000000,6365.000000,6365.000000
mean,37.657816,375.935271,9.519246,0.022203,0.022499
std,35.525809,1306.362110,39.364848,0.031464,0.033638
min,1.000000,58.000000,0.000000,0.000000,0.000000
25%,10.000000,75.000000,0.000000,0.000000,0.000000
50%,29.000000,112.000000,2.000000,0.012500,0.010417
75%,57.000000,233.000000,6.000000,0.030675,0.031396
max,429.000000,40064.000000,1264.000000,0.391892,0.391892


In [45]:
top.sort_values('imp_pcnt')

,ref_term,imp_pcnt,imp_cnt,click_cnt,keyword_ctr,mean_keyword_product_ctr
10569,저스트댄스,1,75,1,0.013333,0.013333
14111,나스,1,83,0,0.000000,0.000000
187,턱걸이,1,176,12,0.068182,0.068182
14234,s1000rr,1,127,2,0.015748,0.015748
1664,수석,1,190,12,0.063158,0.063158
...,...,...,...,...,...,...
864,엔지니어드가먼츠,291,4187,30,0.007165,0.005598
6,아이폰,298,40064,950,0.023712,0.025524
2161,갤럭시,304,8520,267,0.031338,0.025431
999,국내배송,326,2129,57,0.026773,0.032668


- 노출수 기준 상위 5%로 나누는건 노출 상품 수가 적어서 좋은 분류 방법 x

In [46]:
#impression product count가 상위 25% 안에 들어가는 키워드 vs 안들어가는 키워드
top = df[df['imp_pcnt'] >= df['imp_pcnt'].quantile(0.75)]
no_top = df[df['imp_pcnt'] < df['imp_pcnt'].quantile(0.75)]

In [47]:
top

,ref_term,imp_pcnt,imp_cnt,click_cnt,keyword_ctr,mean_keyword_product_ctr
0,샤넬,153,12049,174,0.014441,0.019016
2,갤럭시z플립,65,4288,200,0.046642,0.040344
3,신세계백화점 상품,36,105,7,0.066667,0.035301
6,아이폰,298,40064,950,0.023712,0.025524
7,보이져,12,957,21,0.021944,0.051239
...,...,...,...,...,...,...
115891,투바투 시그 교환,11,11,0,0.000000,0.000000
115997,시나모롤 룸,9,9,0,0.000000,0.000000
116776,여성용 진동,8,8,0,0.000000,0.000000
117083,캐시미어 뷔스티에,8,8,0,0.000000,0.000000


In [48]:
#impression product count가 상위 5% 안에 들어가는 키워드 vs 안들어가는 키워드
top = df[df['imp_pcnt'] >= df['imp_pcnt'].quantile(0.95)]
no_top = df[df['imp_pcnt'] < df['imp_pcnt'].quantile(0.95)]

In [50]:
top.sort_values('imp_pcnt')

,ref_term,imp_pcnt,imp_cnt,click_cnt,keyword_ctr,mean_keyword_product_ctr
14538,떨스데이,30,68,0,0.000000,0.000000
49170,미젤로 미션,30,30,0,0.000000,0.000000
36865,투웨이 후드집업,30,46,0,0.000000,0.000000
11203,웨이션 시그 포카,30,30,0,0.000000,0.000000
33478,이레귤러 재현,30,43,0,0.000000,0.000000
...,...,...,...,...,...,...
864,엔지니어드가먼츠,291,4187,30,0.007165,0.005598
6,아이폰,298,40064,950,0.023712,0.025524
2161,갤럭시,304,8520,267,0.031338,0.025431
999,국내배송,326,2129,57,0.026773,0.032668


In [52]:
#impression product count가 상위 5% 안에 들어가는 키워드 vs 안들어가는 키워드
top = df[df['imp_pcnt'] >= df['imp_pcnt'].quantile(0.95)]
no_top = df[df['imp_pcnt'] < df['imp_pcnt'].quantile(0.95)]

(124676, 6)

In [51]:
df[df['mean_keyword_product_ctr'] > 0]

,ref_term,imp_pcnt,imp_cnt,click_cnt,keyword_ctr,mean_keyword_product_ctr
0,샤넬,153,12049,174,0.014441,0.019016
1,빅톤,3,454,17,0.037445,0.058893
2,갤럭시z플립,65,4288,200,0.046642,0.040344
3,신세계백화점 상품,36,105,7,0.066667,0.035301
4,본체 i5,3,3,2,0.666667,0.666667
...,...,...,...,...,...,...
124446,g pro x superlight,1,1,1,1.000000,1.000000
124568,공기청정기 번개페이,1,1,1,1.000000,1.000000
124569,유니파잉,1,1,1,1.000000,1.000000
124570,인켈오디오셋트,1,1,1,1.000000,1.000000


In [36]:
more20 = df[(df['imp_pcnt'] >= 20) & (df['search_cnt'] >= 20)]
more20.describe()

,search_cnt,imp_pcnt,imp_nonad_pcnt,mean_keyword_product_ctr,imp_cnt,click_cnt,keyword_ctr
count,2266.000000,2266.000000,2266.000000,2266.000000,2266.000000,2266.000000,2266.000000
mean,311.214475,52.653575,1567.512798,0.020730,709.615622,17.851280,0.020976
std,866.314294,39.851009,3724.677199,0.026395,2104.556722,63.380943,0.025286
min,20.000000,20.000000,2.000000,0.000000,20.000000,0.000000,0.000000
25%,31.000000,27.000000,187.250000,0.001251,93.000000,1.000000,0.002742
50%,60.500000,39.000000,502.500000,0.011905,174.000000,2.000000,0.014286
75%,187.000000,64.000000,1337.000000,0.030536,441.000000,8.000000,0.029993
max,12283.000000,429.000000,57149.000000,0.349206,40064.000000,1264.000000,0.384615


In [38]:
more20.to_csv('more20.csv', index=False)

In [45]:
more20.sort_values('search_cnt', ascending=False)

,ref_term,search_cnt,imp_pcnt,imp_nonad_pcnt,mean_keyword_product_ctr,imp_cnt,click_cnt,keyword_ctr
6,아이폰,12283,298,23505,0.025524,40064,950,0.023712
145,나이키,10496,267,57149,0.020068,31881,566,0.017754
28,스톤아일랜드,9445,145,15827,0.025010,21797,530,0.024315
2043,아이패드,9437,67,11511,0.034057,13695,560,0.040891
219,에어팟 프로,9304,32,6638,0.029408,12846,236,0.018371
...,...,...,...,...,...,...,...,...
31730,맥케이지 딕슨 금장,20,23,63,0.000000,55,0,0.000000
9715,닥터지블랙스네일크림,20,47,169,0.003040,104,1,0.009615
9711,블루투스동글이,20,51,125,0.026144,72,2,0.027778
9692,네일젤,20,24,666,0.062500,35,2,0.057143


In [50]:
# iphone = content_position_ctr('\''+ '아이폰' +'\'')
# iphone['content_position'] = iphone['content_position'].astype(int)
iphone.sort_values('content_position').to_csv("iphone.csv", index=False)

In [52]:
iphone[iphone['ctr'] <= 0.025524].sort_values('content_position')

,content_position,ctr
309,0,0.012137
313,5,0.009232
206,10,0.003793
303,15,0.012172
207,25,0.016875
...,...,...
205,545,0.000000
395,550,0.000000
85,555,0.000000
92,560,0.000000


In [62]:
iphone2 = keyword_content_ctr('\'' + '아이폰' + '\'')

In [64]:
iphone2.sort_values('ctr', ascending=False).to_csv('iphone_content_ctr.csv', index=False)

In [53]:
# ctr by content position with given keyword(ref_term)
def content_position_ctr(keyword):
    q = f'''
    SELECT i.content_position,
           count(c.imp_id)/count(i.imp_id)::float AS ctr
    FROM
      (SELECT imp_id,
              ad_set_product_id,
              bid_price,
              content_position
       FROM bun_log_db.api_event_type_impression_ad
       WHERE YEAR||MONTH||DAY = '20210215'
         AND page_id = '검색결과'
         AND trim(ref_term) = {keyword}) i
    LEFT JOIN
      (SELECT imp_id
       FROM bun_log_db.api_event_type_click_ad
       WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
    GROUP BY 1
    '''
    return pd.read_sql(q, con=bun_dw)

In [61]:
def keyword_content_ctr(keyword):
    q = f'''
        SELECT i.content_id,
               p.name,
               count(c.imp_id) as click_cnt,
               count(i.imp_id) as imp_cnt,
               count(c.imp_id)/count(i.imp_id)::float AS ctr
        FROM
          (SELECT imp_id,
                  content_id
           FROM bun_log_db.api_event_type_impression_ad
           WHERE YEAR||MONTH||DAY = '20210215'
             AND page_id = '검색결과'
             AND trim(ref_term) = {keyword}) i
        JOIN service1_quicket.product_info p ON i.content_id = p.id
        LEFT JOIN
          (SELECT imp_id
           FROM bun_log_db.api_event_type_click_ad
           WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
        GROUP BY 1, 2
    '''
    return pd.read_sql(q, con=bun_dw)

In [15]:
def keyword_product_bid(keyword):
    q = f'''
        SELECT p.name,
               count(i.imp_id) AS imp_cnt,
               listagg(b.keyword || b.bid_price, ',')
        FROM
          (SELECT imp_id,
                  ad_set_product_id
           FROM bun_log_db.api_event_type_impression_ad
           WHERE YEAR||MONTH||DAY = '20210215'
             AND page_id = '검색결과'
             AND trim(ref_term) = {keyword}) i
        JOIN service1_quicket.ad_set_product a ON i.ad_set_product_id = a.id
        JOIN service1_quicket.ad_set_product_bid_keyword b ON a.id = b.ad_set_product_id
        JOIN service1_quicket.product_info p ON a.pid = p.id
        GROUP BY 1
        ORDER BY 2 DESC
    '''
    return pd.read_sql(q, con=bun_dw)

In [17]:
frostj = keyword_product_bid('\'' + '프로스트제이' + '\'')

In [19]:
frostj.to_csv('프로스트제이.csv', index=False)

## 비광고 상품 

In [59]:
q = '''
SELECT i.ref_term,
       count(DISTINCT i.content_id) AS imp_nonad_pcnt,
       count(i.imp_id) AS non_ad_imp_cnt,
       count(c.imp_id) AS non_ad_click_cnt,
       non_ad_click_cnt/non_ad_imp_cnt::float AS non_ad_ctr
FROM
  (SELECT imp_id,
          content_id,
          trim(ref_term) AS ref_term
   FROM bun_log_db.app_event_type_impression
   WHERE YEAR||MONTH||DAY = '20210215'
     AND page_id = '검색결과'
     AND ref_source IS NULL) i
LEFT JOIN
  (SELECT imp_id
   FROM bun_log_db.app_event_type_view
   WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
GROUP BY 1
HAVING count(i.imp_id) > 0
'''
daily_nonad = pd.read_sql(q, con=bun_dw)

In [61]:
q = '''
SELECT i.ref_term,
       i.content_id,
       count(i.imp_id) AS non_ad_imp_cnt,
       count(c.imp_id) AS non_ad_click_cnt,
       non_ad_click_cnt/non_ad_imp_cnt::float AS non_ad_ctr
FROM
  (SELECT imp_id,
          content_id,
          trim(ref_term) AS ref_term
   FROM bun_log_db.app_event_type_impression
   WHERE YEAR||MONTH||DAY = '20210215'
     AND page_id = '검색결과'
     AND ref_source IS NULL) i
LEFT JOIN
  (SELECT imp_id
   FROM bun_log_db.app_event_type_view
   WHERE YEAR||MONTH||DAY = '20210215') c ON i.imp_id = c.imp_id
GROUP BY 1, 2
HAVING count(i.imp_id) > 0
'''
keyword_product_ctr_nonad = pd.read_sql(q, con=bun_dw)

In [20]:
# daily_nonad.to_csv('daily_nonad.csv', index=False)
# keyword_product_ctr_nonad.to_csv('keyword_product_ctr_nonad.csv', index=False)
daily_nonad = pd.read_csv('daily_nonad.csv')

In [62]:
nonad_df = pd.merge(daily_nonad, keyword_product_ctr_nonad.groupby('ref_term', as_index=False)['non_ad_ctr'].mean(), on='ref_term')
nonad_df.columns = ['ref_term', 'imp_nonad_pcnt', 'nonad_imp_cnt', 'nonad_click_cnt', 'nonad_keyword_ctr', 'non_ad_mean_keyword_product_ctr']

In [63]:
nonad_df

,ref_term,imp_nonad_pcnt,nonad_imp_cnt,nonad_click_cnt,nonad_keyword_ctr,non_ad_mean_keyword_product_ctr
0,!i5 9400,264,306,2,0.006536,0.007576
1,!국내발송!,48,71,2,0.028169,0.026042
2,!국내발송! 샤넬,53,57,10,0.175439,0.169811
3,#235,59,62,4,0.064516,0.067797
4,#K7,119,122,1,0.008197,0.008403
...,...,...,...,...,...,...
531535,힛스,8,9,0,0.000000,0.000000
531536,힛홉,1,1,1,1.000000,1.000000
531537,힢마,56,217,20,0.092166,0.069460
531538,힢마 만쥬,6,6,3,0.500000,0.500000


In [68]:
data = pd.merge(nonad_df, df, on='ref_term', how='outer')[['ref_term', 'imp_nonad_pcnt', 'imp_pcnt', 'nonad_keyword_ctr', 'non_ad_mean_keyword_product_ctr', 'keyword_ctr', 'mean_keyword_product_ctr']]

In [73]:
# na_ : no ad
# a_ : ad 
data.columns = ['ref_term', 'na_imp_pcnt', 'a_imp_pcnt', 'na_keyword_ctr', 'na_mean_keyword_product_ctr', 'a_keyword_ctr', 'a_mean_keyword_product_ctr']

In [75]:
data.describe()

,na_imp_pcnt,a_imp_pcnt,na_keyword_ctr,na_mean_keyword_product_ctr,a_keyword_ctr,a_mean_keyword_product_ctr
count,531540.000000,124676.000000,531540.000000,531540.000000,124676.000000,124676.000000
mean,73.227900,7.426193,0.134151,0.128629,0.032035,0.031796
std,356.434502,13.100567,0.217641,0.215998,0.133377,0.133107
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,16.000000,3.000000,0.050000,0.044118,0.000000,0.000000
75%,54.000000,8.000000,0.153700,0.142857,0.000000,0.000000
max,57149.000000,429.000000,1.000000,1.000000,1.000000,1.000000
